#installing required package

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.9 MB/s eta 0:00:00


#importing essential packasges

In [2]:
import optuna
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical

#loading mnist dataset and Normalizing it

In [3]:
# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


# creating Model architecture and hyper parameters search space

In [5]:
def create_model(trial):
    # Suggest hyperparameters
    n_layers = trial.suggest_int('n_layers', 1, 3)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Build the model
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    for i in range(n_layers):
        num_hidden = trial.suggest_int('n_units_l{}'.format(i), 32, 128)
        model.add(Dense(num_hidden, activation=activation))
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# train the model on data

In [7]:
def objective(trial):
    model = create_model(trial)
    model.fit(x_train, y_train, epochs=10, batch_size=128, verbose=0, validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]  # We aim to maximize accuracy

#testing the model on trails

In [8]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10) # because of gpu computing i set n_trials = 10 , you can do it like 100

[I 2024-07-31 19:51:47,161] A new study created in memory with name: no-name-b78f2fbc-3eb0-4341-b20e-21e21647af31
<ipython-input-5-0aa35023674a>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2024-07-31 19:52:05,785] Trial 0 finished with value: 0.9218000173568726 and parameters: {'n_layers': 2, 'activation': 'relu', 'optimizer': 'adam', 'learning_rate': 3.0123815745827146e-05, 'n_units_l0': 69, 'n_units_l1': 73}. Best is trial 0 with value: 0.9218000173568726.
[I 

In [9]:
print("Best trial:")
trial = study.best_trial

print("Value: {}".format(trial.value))

print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
Value: 0.9767000079154968
Params: 
    n_layers: 1
    activation: sigmoid
    optimizer: rmsprop
    learning_rate: 0.005140608449720418
    n_units_l0: 94
